# Table of Content
- [9.1 Putting a Wrapper Around a Function](#9.1)
- [9.4 Defining a Decorator That Takes Arguments](#9.4)
- [9.6 Defining a Decorator That Takes an Optional Argument](#9.6)
- [9.10 Applying Decorators to Class and Static Methods](#9.10)
- [9.11 Writing Decorators That Add Arguments to Wrapped Functions](#9.11)
- [9.13 Using a Metaclass to Control Instance Creation](#9.13)

---
## <a name="9.1"></a> 9.1 Putting a Wrapper Around a Function

### Solution

***`@wraps(func)`***

In [1]:
import time
from functools import wraps

def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

@timethis
def countdown(n):
    while n > 0:
        n -= 1
        
countdown(10000)

countdown 0.0013880729675292969


---
## <a name="9.4"></a> 9.4 Defining a Decorator That Takes Arguments
### Solution

In [2]:
from functools import wraps
import logging

def logged(level, name=None, message=None):
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

@logged(logging.CRITICAL, None, 'Example')
def add(x, y):
    return x + y

add(1, 2)

Example


3

---
## <a name="9.6"></a> 9.6 Defining a Decorator That Takes an Optional Argument


### Solution

In [3]:
from functools import wraps, partial
import logging

def logged(func=None, *, level=logging.DEBUG, name=None, messsage=None):
    if func is None:
        return partial(logged, level=level, name=None, messsage=messsage)
    
    logname = name if name else func.__name__
    log = logging.getLogger(logname)
    logmsg = message if messsage else func.__name__
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        log.log(level, logmsg)
        return func(*args, **kwargs)
    return wrapper

In [4]:
@logged
def add(x, y):
    return x + y

@logged(level=logging.CRITICAL, name='example')
def spam():
    print('spam!')

In [5]:
add(1, 2)

3

In [6]:
spam()

spam


spam!


### Discussion

The following code is equvalent to the code above using decorators

In [7]:
def add(x, y):
    return x + y
add = logged(add)

def spam():
    print('Spam!')
spam = logged(level=logging.CRITICAL, name='example')(spam)

---

## <a name="9.10"></a> 9.10 Applying Decorators to Class and Static Methods

### Solution
***Make sure that your decorators are applied before @classmethod or @staticmethod***  
e.g.
```python
class Class:
    @your_decorator
    @staticmethod
    def func():
        pass
```

### Discussion
@classmethod and @staticmethod don't actually create objects that are directly callable.  
They create special descriptor objects instead  
Thus, if you try to use them like functions in another decorator, the decorator will crash.

---
## <a name="9.11"></a> 9.11 Writing Decorators That Add Arguments to Wrapped Functions

### Solution

This can be used when certain optional arguements would be used across multiple functions.

In [8]:
from functools import wraps
import inspect

def optional_debug(func):
    # Avoid redifinition of argument debug
    if 'debug' in list(inspect.signature(func).parameters.keys()):
        raise TypeError('debug argument alread defined')
        
    @wraps(func)
    def wrapper(*args, debug=False, **kwargs):
        if debug:
            print('Calling', func.__name__)
        return func(*args, **kwargs)
    
    # Correct signature
    sig = inspect.signature(func)
    parms = list(sig.parameters.values())
    parms.append(inspect.Parameter('debug',
                                   inspect.Parameter.KEYWORD_ONLY,
                                   default=False))
    wrapper.__signature__ = sig.replace(parameters=parms)
    return wrapper

In [9]:
@optional_debug
def add(x, y):
    return x + y

@optional_debug
def mul(x, y):
    return x * y

In [10]:
add(1, 2)

3

In [11]:
add(1, 2, debug=True)

Calling add


3

---
## <a name="9.13"></a> 9.13 Using a Metaclass to Control Instance Creation

### Solution
- Signleton

In [12]:
class Singleton(type):
    def __init__(self, *args, **kwargs):
        self.__instance = None
        super().__init__(*args, **kwargs)
        
    def __call__(self, *args, **kwargs):
        if self.__instance is None:
            self.__instance = super().__call__(*args, **kwargs)
            return self.__instance
        else:
            return self.__instance
        
class Spam(metaclass=Singleton):
    def __init__(self):
        print('Creating Spam')

In [13]:
a = Spam()

Creating Spam


In [14]:
b = Spam()

In [15]:
a is b

True

### Discussion
- Cached

In [16]:
import weakref

class Cached(type):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.__cache = weakref.WeakValueDictionary()
        
    def __call__(self, *args):
        if args in self.__cache:
            return self.__cache[args]
        else:
            obj = super().__call__(*args)
            self.__cache[args] = obj
            return obj
        
class Spam(metaclass=Cached):
    def __init__(self, name):
        print('Creating Spam{!r}'.format(name))
        self.name = name

In [17]:
a = Spam('aaa')
b = Spam('bbb')
c = Spam('aaa')

Creating Spam'aaa'
Creating Spam'bbb'


In [18]:
a is c

True

In [19]:
a is b

False